In [11]:
from pycaret.datasets import get_data
from pycaret.classification import *
import pandas as pd
from pandas_profiling import ProfileReport


data = pd.read_csv("C:/Users/DELL/SM4_MLOPS/SleepHealth/data/Sleep_health_and_lifestyle_dataset.csv")
Dataset = data.iloc[:, 1:]
Dataset

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,Not
1,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,Not
2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,Not
3,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...
369,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


In [12]:
Dataset.dtypes

Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

In [16]:
s = setup(Dataset, target = 'Sleep Disorder', session_id = 123, categorical_features = ['Gender', 'Occupation', 'BMI Category', 'Blood Pressure'])

,Description,Value
0,Session id,123
1,Target,Sleep Disorder
2,Target type,Multiclass
3,Target mapping,"Insomnia: 0, Not: 1, Sleep Apnea: 2"
4,Original data shape,"(374, 12)"
5,Transformed data shape,"(374, 45)"
6,Transformed train set shape,"(261, 45)"
7,Transformed test set shape,"(113, 45)"
8,Numeric features,7
9,Categorical features,4


In [18]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:31:21
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.9004,0.0000,0.9004,0.9093,0.8949,0.8238,0.8314,0.0810
ridge,Ridge Classifier,0.8964,0.0000,0.8964,0.9064,0.8919,0.8171,0.8252,0.0810
lr,Logistic Regression,0.8926,0.0000,0.8926,0.9055,0.8900,0.8119,0.8203,0.2600
rf,Random Forest Classifier,0.8926,0.0000,0.8926,0.9030,0.8872,0.8110,0.8188,0.1980
xgboost,Extreme Gradient Boosting,0.8889,0.0000,0.8889,0.8978,0.8832,0.8048,0.8122,0.1860
gbc,Gradient Boosting Classifier,0.8887,0.0000,0.8887,0.8997,0.8836,0.8048,0.8125,0.3460
lightgbm,Light Gradient Boosting Machine,0.8812,0.0000,0.8812,0.8922,0.8760,0.7920,0.8002,0.2480
lda,Linear Discriminant Analysis,0.8811,0.0000,0.8811,0.8932,0.8761,0.7885,0.7983,0.0870
et,Extra Trees Classifier,0.8695,0.0000,0.8695,0.8854,0.8655,0.7743,0.7836,0.2190
knn,K Neighbors Classifier,0.8623,0.0000,0.8623,0.8724,0.8574,0.7574,0.7639,0.1090


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [19]:
model = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8889,0.0000,0.8889,0.8877,0.8855,0.8039,0.8058
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9615,0.0000,0.9615,0.9679,0.9611,0.9297,0.9323
3,0.9231,0.0000,0.9231,0.9321,0.9216,0.8598,0.8684
4,0.8846,0.0000,0.8846,0.9090,0.8724,0.7963,0.8117
5,0.8077,0.0000,0.8077,0.8029,0.8031,0.6579,0.6599
6,0.9231,0.0000,0.9231,0.9423,0.9258,0.8693,0.8760
7,0.9615,0.0000,0.9615,0.9679,0.9615,0.9335,0.9359
8,0.8462,0.0000,0.8462,0.8718,0.8147,0.7143,0.7454


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [20]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [21]:
finalize_model(model)

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['Age', 'Sleep Duration',
                                             'Quality of Sleep',
                                             'Physical Activity Level',
                                             'Stress Level', 'Heart Rate',
                                             'Daily Steps'],
                                    transformer=SimpleImputer(add_indicat...
                                                                    'Blood '
                                                                    'Pressure'],
                                                              drop_invariant=False,
                                                              handle_missing='return_nan',
                                                              handle_unknown='value',
                                                              return_df=True,
                                                              use_cat_names=True,
                                                              verbose=0))),
                ('clean_column_names',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                ('actual_estimator',
                 GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)

In [22]:
save_model(model, 'SleepHealth_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Age', 'Sleep Duration',
                                              'Quality of Sleep',
                                              'Physical Activity Level',
                                              'Stress Level', 'Heart Rate',
                                              'Daily Steps'],
                                     transformer=SimpleImputer(add_indicat...
                                                                     'Category',
                                                                     'Blood '
                                                                     'Pressure'],
             